In [4]:
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src/data')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src/utils')))

import cv_utils
import file_utils as futils
import train_utils as tu


In [40]:
import torchvision
from  torchvision.models.detection import ssd, ssd300_vgg16, SSD300_VGG16_Weights, fasterrcnn_resnet50_fpn
from torchvision.io.image import decode_image
import torch


In [54]:
detector = ssd300_vgg16(num_classes=1)


In [55]:
img = decode_image('/home/ezaborshchikov/hse_mc/hse_dl_project/dataset/yolo/images/test/img_58.jpg')
img.shape

torch.Size([3, 480, 480])

In [56]:
img = img.unsqueeze(dim=0)


In [57]:
img = img / 255.0

In [58]:
detector.eval()

SSD(
  (backbone): SSDFeatureExtractorVGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=

In [60]:
detector(img)

RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [41]:
object_detection_model = fasterrcnn_resnet50_fpn(pretrained=True, progress=False)

object_detection_model.eval();

/home/ezaborshchikov/hse_mc/hse_dl_project/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ezaborshchikov/hse_mc/hse_dl_project/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /home/ezaborshchikov/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [42]:
p = object_detection_model(img)

In [46]:
detector.eval()

SSD(
  (backbone): SSDFeatureExtractorVGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=

In [47]:
detector(img)

[{'boxes': tensor([[277.1865, 130.9042, 342.7343, 184.6990],
          [157.7070, 243.6781, 194.3656, 279.9762],
          [174.1988,  90.0836, 290.8433, 129.9689],
          [117.9178, 239.6721, 176.2448, 274.7505],
          [361.1108,   0.0000, 445.7795,  19.4573],
          [181.0654, 247.9480, 213.3205, 286.1430],
          [119.8722, 154.4980, 480.0000, 218.4832],
          [279.1771,  79.0645, 352.9485, 162.7504],
          [244.0425,  87.4798, 322.3822, 138.2124],
          [136.4043,  93.4422, 231.5616, 132.1182],
          [108.9051,  35.4008, 205.9624,  69.3723],
          [208.7143, 307.5201, 327.0450, 329.9831],
          [203.7608, 182.5719, 237.5440, 228.6549],
          [132.6407,  69.2285, 204.8975, 113.7910],
          [222.7474, 387.3967, 394.0771, 480.0000],
          [ 97.5633, 241.4240, 150.8410, 274.6300],
          [253.7977, 235.6119, 285.7553, 282.4509],
          [201.2866, 254.5941, 234.1696, 288.8758],
          [221.2146, 230.6725, 319.0284, 288.9338],
   